#Use a proper tensorflow version
Our model was build on TF2.14 and it does cause some compatibility issues with TF2.17 so make sure you use the right version

In [1]:
#Check tf version
import tensorflow as tf
#print(tf.__version__)

#The following force you to use tensorflow 2.14 but you need to restart. You may comment it if you know you are using 2.14
##Make sure you down grade!! otherwise, some part will fail due to compatibility.
#!pip3 install --upgrade tensorflow==2.14.0
print(tf.__version__)

C:\Users\asheikhz\AppData\Local\anaconda3\envs\tf\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


2.10.0


#Get the dataset

In [2]:
# mount google drive on your runtime using and authorization code.
# more details here: https://colab.research.google.com/notebooks/io.ipynb
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
#Set directories correctly

In [2]:
# Path to the directory containing the project files (CHANGE THIS PATH TO THE DIRECTORY ON YOUR COMPUTER)
import os
PROJECT_ROOT_DIR = os.getcwd() + '/'

# Path to the directory containing the dataset relative to project file
DATA_DIR = 'data\\'

#path to the directory you want to use for saving models relative to the project file
MODEL_DIR = r'C:\Users\asheikhz\Desktop\Assignment2'

In [3]:
# Funciton for loading the dataset
# Code from advml-traffic-sign (https://github.com/inspire-group/advml-traffic-sign)
def load_dataset_GTSRB(n_channel=3, train_file_name=None):
    """
    Load GTSRB data as a (datasize) x (channels) x (height) x (width) numpy
    matrix. Each pixel is rescaled to the range [0,1].
    """

    def load_pickled_data(file, columns):
        """
        Loads pickled training and test data.

        Parameters
        ----------
        file    : string
                          Name of the pickle file.
        columns : list of strings
                          List of columns in pickled data we're interested in.

        Returns
        -------
        A tuple of datasets for given columns.
        """

        with open(file, mode='rb') as f:
            dataset = pickle.load(f)
        return tuple(map(lambda c: dataset[c], columns))

    def preprocess(x, n_channel):
        """
        Preprocess dataset: turn images into grayscale if specified, normalize
        input space to [0,1], reshape array to appropriate shape for NN model
        """

        if n_channel == 3:
            # Scale features to be in [0, 1]
            x = (x / 255.).astype(np.float32)
        else:
            # Convert to grayscale, e.g. single Y channel
            x = 0.299 * x[:, :, :, 0] + 0.587 * x[:, :, :, 1] + \
                0.114 * x[:, :, :, 2]
            # Scale features to be in [0, 1]
            x = (x / 255.).astype(np.float32)
            x = x[:, :, :, np.newaxis]
        return x

    # Load pickle dataset
    if train_file_name is None:
        x_train, y_train = load_pickled_data(
            PROJECT_ROOT_DIR + DATA_DIR + 'train.p', ['features', 'labels'])
    else:
        x_train, y_train = load_pickled_data(
            PROJECT_ROOT_DIR + DATA_DIR + train_file_name, ['features', 'labels'])
    x_val, y_val = load_pickled_data(
        PROJECT_ROOT_DIR + DATA_DIR + 'valid.p', ['features', 'labels'])
    x_test, y_test = load_pickled_data(
        PROJECT_ROOT_DIR + DATA_DIR + 'test.p', ['features', 'labels'])

    # Preprocess loaded data
    x_train = preprocess(x_train, n_channel)
    x_val = preprocess(x_val, n_channel)
    x_test = preprocess(x_test, n_channel)
    return x_train, y_train, x_val, y_val, x_test, y_test

In [4]:
import pickle
import numpy as np
# Load the images and labels. These images are RGB so we have 3 channels
imgs_train, labels_train, imgs_val, labels_val, imgs_test, labels_test = load_dataset_GTSRB(n_channel=3)

#Load the built model from Assignment 1 (Already on blackboard)

In [5]:
## Load the saved the model
from tensorflow.keras.models import load_model



#load our already trained model
# Your earlier directories should change to match this one
#You should be able to load the model once trained. The following line is used for that.
model = load_model(MODEL_DIR +'\VGG_best.h5')

#Some constants that you might need
It is not neccessary that you will use all

In [6]:
# Set constants (GTSRB)
NUM_LABELS = 43                             # Number of labels or classes for classification
BATCH_SIZE = 128                            # Size of batch
HEIGHT = 32                                 # Height of input image
WIDTH = 32                                  # Width of input image
N_CHANNEL = 3                               # Number of channels
OUTPUT_DIM = 43                             # Number of output dimension

# Set training hyperparameters
NUM_EPOCH = 50                            # Number of epoch to train
LR = 0.0002                                 # Learning rate
RBW = True #restore best weights
PATIENCE = 5# how many epochs between improvements
sample_size = 100
alpha = 0.001
INPUT_SHAPE = (HEIGHT, WIDTH, N_CHANNEL)  # Input shape of model
IMG_SHAPE = (HEIGHT, WIDTH, N_CHANNEL)

# setting up labels

In [7]:

from tensorflow.keras.utils import to_categorical

#Convert the labels to one-hot encoding (to input to the models)

#Use the following sub-test dataset for your certified training
imgs_sub_test = imgs_test[0:500,:,:,:].copy()
labels_adv = labels_test[0:500].copy()




labels_train_cat = to_categorical(labels_train, NUM_LABELS)
labels_test_cat = to_categorical(labels_test, NUM_LABELS)
labels_val_cat = to_categorical(labels_val, NUM_LABELS)

#for testing adversarial inputs
labels_adv_cat = to_categorical(labels_adv,NUM_LABELS)

print('Labels train shape: {}'.format(labels_train.shape))
print('Labels train catagorical shape: {}\n'.format(labels_train_cat.shape))
print('Labels Adver shape: {}'.format(labels_adv.shape))
print('Labels Adver catagorical shape: {}'.format(labels_adv_cat.shape))

print('Imgs Adver shape: {}'.format(imgs_sub_test.shape))

Labels train shape: (34799,)
Labels train catagorical shape: (34799, 43)

Labels Adver shape: (500,)
Labels Adver catagorical shape: (500, 43)
Imgs Adver shape: (500, 32, 32, 3)


#Install ART if needed

In [10]:
#Install ART
!pip3 install adversarial-robustness-toolbox

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 44.2 MB/s eta 0:00:00


#Some imports.
You might need more

In [8]:
from art.estimators.classification import TensorFlowV2Classifier

import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt

C:\Users\asheikhz\AppData\Local\anaconda3\envs\tf\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


##Task 2

Now, let's do some privacy attacks. We will focus on Membership inference attacks. This task will implement label-only attacks, while the next will implement shadow modeling. As discussed in class, the label-only attack is developed based on the fact that it costs an attacker so much to generate a successful adversarial sample for a member sample. In contrast, the cost will be much less for a non-member sample. Thus, it uses HopSkipJump to generate a successful adversarial sample and then measure the perturbation using the L2 norm. If the perturbation is above a threshold, it will be classified as a member; if not, it will be classified as a non-member. Hence, we will follow the following steps to perform the attack (Your reference code is [this notebook](https://github.com/Trusted-AI/adversarial-robustness-toolbox/blob/main/notebooks/label_only_membership_inference.ipynb)):

1-	Generate a dataset (already given): The training and testing datasets are merged into one, with the training classified as members and the testing as non-members. We picked 1500 samples from this dataset to build our attack model.


2-	Calculate the threshold (to be implemented by you): Your task here is to use existing implementations in ART to calculate/calibrate the threshold. All parameters are given to you in the code (in comments).

3-	Infer the data: Now, use the provided 1500 samples to predict whether they are members. Then, compare your performance to the ground truth and present the results.


One parameter you will need to set is the maximum number of queries (used in step 2). Set this parameter to 5 and 10. All that is needed is to answer the questions below.


##Note on resources:
It takes time to generate the adversarial examples. A good GPU computer will take you around 1-3 hours per round. Thus, plan your time accordingly.


#Generate a dataset

In [9]:
##Some variables to set  number of samples used to calibrate distance threshold
x = np.concatenate([imgs_train, imgs_test]) #Merge train and test dataset
y = np.concatenate([labels_train_cat, labels_test_cat]) #Merge train and test dataset
training_sample = np.array([1] * len(imgs_train) + [0] * len(imgs_test)) #Class label for the attack model training. What does it say?

from numpy.random import choice

##Random samples to be used for training our attack model
# evaluation data
n = 1500
eval_data_idx = choice(len(x), n) #choose 1500 of the samples
x_eval, y_eval = x[eval_data_idx], y[eval_data_idx] #creating your training dataset for the attack model
eval_label = training_sample[eval_data_idx] #creating your training label for the attack model

#Calculate the threshold

In [10]:
import torch.optim as optim
from art.attacks.inference.membership_inference import LabelOnlyDecisionBoundary
from sklearn.metrics import accuracy_score
from art.estimators.classification.pytorch import PyTorchClassifier
from torch import nn
# criterion = nn.CrossEntropyLoss()
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
loss_object = tf.keras.losses.CategoricalCrossentropy(from_logits=False)
art_model = TensorFlowV2Classifier(
    model=model,
    nb_classes=NUM_LABELS,
    input_shape=INPUT_SHAPE,
    loss_object=loss_object,
    optimizer=optimizer,
    clip_values=(0, 1)
)

# Members and Non-members accuracy at queries = 5
mia_label_only_unsupervised = LabelOnlyDecisionBoundary(art_model)
mia_label_only_unsupervised.calibrate_distance_threshold_unsupervised(top_t=20, num_samples=200, max_queries=5, verbose=True, batch_size=256)

HopSkipJump:   0%|                                                                              | 0/38 [00:00<?, ?it/s]C:\Users\asheikhz\AppData\Local\anaconda3\envs\tf\lib\site-packages\art\attacks\evasion\hop_skip_jump.py:606: RuntimeWarning: invalid value encountered in divide
  rnd_noise = (eval_samples - current_sample) / delta
HopSkipJump:  11%|███████▎                                                              | 4/38 [00:27<04:17,  7.57s/it]C:\Users\asheikhz\AppData\Local\anaconda3\envs\tf\lib\site-packages\art\attacks\evasion\hop_skip_jump.py:626: RuntimeWarning: invalid value encountered in divide
  result = grad / np.linalg.norm(grad)
HopSkipJump: 100%|█████████████████████████████████████████████████████████████████████| 29/29 [02:28<00:00,  5.11s/it]


In [11]:
pred_label_unsupervised = mia_label_only_unsupervised.infer(x_eval, y_eval)

HopSkipJump: 100%|███████████████████████████████████████████████████████████████| 1500/1500 [4:05:05<00:00,  9.80s/it]


In [12]:
members_accuracy = accuracy_score(eval_label[eval_label == 1], pred_label_unsupervised[eval_label == 1])
non_members_accuracy = accuracy_score(eval_label[eval_label == 0], pred_label_unsupervised[eval_label == 0])
overall_accuracy = accuracy_score(eval_label, pred_label_unsupervised)
print("Members Accuracy: %f" % members_accuracy)
print("Non Members Accuracy: %f" % non_members_accuracy)
print("Overall Accuracy: %f" % overall_accuracy)

Members Accuracy: 0.887569
Non Members Accuracy: 0.152709
Overall Accuracy: 0.688667


In [13]:
mia_label_only_unsupervised = LabelOnlyDecisionBoundary(art_model)
mia_label_only_unsupervised.calibrate_distance_threshold_unsupervised(top_t=20, num_samples=200, max_queries=10, verbose=True, batch_size=256)

HopSkipJump:   2%|█▍                                                                    | 1/48 [00:08<06:37,  8.45s/it]C:\Users\asheikhz\AppData\Local\anaconda3\envs\tf\lib\site-packages\art\attacks\evasion\hop_skip_jump.py:606: RuntimeWarning: invalid value encountered in divide
  rnd_noise = (eval_samples - current_sample) / delta
HopSkipJump:  25%|█████████████████▎                                                   | 12/48 [01:47<05:39,  9.42s/it]C:\Users\asheikhz\AppData\Local\anaconda3\envs\tf\lib\site-packages\art\attacks\evasion\hop_skip_jump.py:626: RuntimeWarning: invalid value encountered in divide
  result = grad / np.linalg.norm(grad)
HopSkipJump: 100%|█████████████████████████████████████████████████████████████████████| 30/30 [03:36<00:00,  7.23s/it]


In [14]:
pred_label_unsupervised = mia_label_only_unsupervised.infer(x_eval, y_eval)

HopSkipJump: 100%|███████████████████████████████████████████████████████████████| 1500/1500 [4:05:01<00:00,  9.80s/it]


In [15]:
# Members and Non-members accuracy at queries = 10
members_accuracy = accuracy_score(eval_label[eval_label == 1], pred_label_unsupervised[eval_label == 1])
non_members_accuracy = accuracy_score(eval_label[eval_label == 0], pred_label_unsupervised[eval_label == 0])
overall_accuracy = accuracy_score(eval_label, pred_label_unsupervised)
print("Members Accuracy: %f" % members_accuracy)
print("Non Members Accuracy: %f" % non_members_accuracy)
print("Overall Accuracy: %f" % overall_accuracy)

Members Accuracy: 0.898537
Non Members Accuracy: 0.133005
Overall Accuracy: 0.691333
